# Deliverable 6.2: Landing MPC with All Four Subsystems

Merge the robust tube MPC for z-dimension with nominal MPC controllers for x, y, and roll subsystems.

In [ ]:
%load_ext autoreload
%autoreload 2

# Get parent directory and add to sys.path
import sys, os
parent_dir = os.path.dirname(os.getcwd())
sys.path.append(parent_dir)

# Require ipympl
%matplotlib widget 

In [ ]:
import time
import numpy as np
from LandMPC.MPCLandControl import MPCLandControl
from src.rocket import Rocket
from src.pos_rocket_vis import *

rocket_obj_path = os.path.join(parent_dir, "Cartoon_rocket.obj")
rocket_params_path = os.path.join(parent_dir, "rocket.yaml")

# Rocket setup
Ts  = 1/20
rocket = Rocket(Ts=Ts, model_params_filepath=rocket_params_path)
rocket.mass = 1.7  # Do not change!!!

# Visualization setup
vis = RocketVis(rocket, rocket_obj_path)
vis.anim_rate = 1

In [ ]:
# Landing maneuver: from (3, 2, 10, 30°) to (1, 0, 3, 0°)
sim_time = 15  # simulation length in seconds

# Initial state: at (3, 2, 10, 30°)
x0 = np.array([0, 0, 0,  # angular velocities (wx, wy, wz)
               0, 0, np.deg2rad(30),  # angles (alpha, beta, gamma/roll)
               0, 0, 0,  # linear velocities (vx, vy, vz)
               3, 2, 10])  # positions (x, y, z)

# Target state: (1, 0, 3, 0°)
x_ref = np.array([0.]*9 + [1., 0., 3.])  # Target: x=1, y=0, z=3

xs, us = rocket.trim(x_ref)
print("Linearization around landing target:")
print("x_ref = ", x_ref)
print("xs = ", xs)
print("us = ", us)

A, B = rocket.linearize(xs, us)

# MPC parameters
H = 2.0  # horizon time

# Create merged landing controller with all four subsystems
mpc = MPCLandControl().new_controller(rocket, Ts, H, x_ref=x_ref)
print("\nMerged landing MPC controller initialized")
print("  - MPCControl_x: nominal MPC for x-position")
print("  - MPCControl_y: nominal MPC for y-position")
print("  - MPCControl_z: robust tube MPC for z-position")
print("  - MPCControl_roll: nominal MPC for roll angle")

## Simulate Landing Maneuver

In [ ]:
# Test with LINEAR model to verify MPC logic
H_sim = H  # Use the same horizon as controller (2s)
t_cl, x_cl, u_cl, t_ol, x_ol, u_ol = rocket.simulate_land(mpc, sim_time, H_sim, x0, method='linear')
vis.animate(t_cl[:-1], x_cl[:,:-1], u_cl, T_ol=t_ol[...,:-1], X_ol=x_ol, U_ol=u_ol)
xs_full = mpc.xs  # Full state equilibrium point
plot_static_states_inputs(t_cl[:-1], x_cl[:,:-1], u_cl, xs_full)

In [ ]:
# Final results for 6_2
print(f"\n===== Deliverable 6_2 Final Results =====")
print(f"Position:")
print(f"  x = {x_cl[9, -1]:.3f} m (target: {x_ref[9]:.1f} m) | error: {abs(x_cl[9, -1] - x_ref[9]):.3f} m")
print(f"  y = {x_cl[10, -1]:.3f} m (target: {x_ref[10]:.1f} m) | error: {abs(x_cl[10, -1] - x_ref[10]):.3f} m")
print(f"  z = {x_cl[11, -1]:.3f} m (target: {x_ref[11]:.1f} m) | error: {abs(x_cl[11, -1] - x_ref[11]):.3f} m")
print(f"\nVelocity:")
print(f"  vx = {x_cl[6, -1]:.3f} m/s")
print(f"  vy = {x_cl[7, -1]:.3f} m/s")
print(f"  vz = {x_cl[8, -1]:.3f} m/s")
print(f"\nOrientation:")
print(f"  alpha = {np.rad2deg(x_cl[3, -1]):.2f}° (target: 0°)")
print(f"  beta = {np.rad2deg(x_cl[4, -1]):.2f}° (target: 0°)")
print(f"  gamma = {np.rad2deg(x_cl[5, -1]):.2f}° (target: 0°)")